- Update: 2022.12.27

# 0. 配置参数

- `dict_file`：需要内部查重的文件的相关信息
    - `file_name`：文件的路径
    - `sheet_name`：工作表名称
    - `head`：表头行数
    - `col_oe`：OE号所在列的列名
    - `col_field`：需要查重的列的列名
- `out`：保存结果的路径

In [1]:
dict_file = {'file_name': '../../file/膨胀壶目录221213.xlsx',
             'sheet_name': 'Sheet1',
             'head': 1,
             'col_oe': '完整OE',
             'col_field': '完整OE'}

out = '../../file/内部查重_' + dict_file['col_field'] + '_NOW().xlsx'

In [2]:
import re

In [3]:
def normalise_oe(oe, test=False):
    oe = str(oe).strip()
    
    if test:
        print(f'oe:\n{oe}')
    
    oe = oe.replace('\u200e', '')
    oe = oe.replace('\n', ';')
    
    oe = re.sub(r'(GM|Subaru|Dodge|Ford)[:：]', '', oe) # 去除品牌号
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    oe = re.sub(r'[\(（].*?[\)）]', '', oe) # 去除括号，注意用非贪婪模式
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    oe = re.sub(r'[\u4e00-\u9fa5]', '', oe) # 去除汉字
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    oe = oe.replace('-', '')
    oe = re.sub(r'\*+[2-9]?', '', oe)
    oe = oe.replace(' ', ';')
    oe = oe.replace('.', ';')
    oe = oe.replace('//', ';')
    oe = re.sub(r'[,，。;；+=/、]', ';', oe)
    
    oe = oe.replace(';;;', ';')
    oe = oe.replace(';;', ';')
    
    if test:
        print()
        print(f'oe:\n{oe}')
    
    return oe.upper()

# oe_1 = 'GM：OE001;Dodge: OE002;OE003（test);OE004(test）;测试OE005;OE006 OE007.OE-008;;;OE009//OE010\n'
# oe_2 = 'OE011,OE012，OE013。OE014；OE015+OE016=OE017/OE018、OE019,；OE020，；'
# oe_3 = 'OE021。；OE022,;OE023，;OE024。;OE025*;OE026*2;OE027*3;OE028**;OE029***;oE030'

# normalise_oe(oe_1 + oe_2 + oe_3, True)

# 1. 处理OE

In [4]:
import pandas as pd

In [5]:
df_data = pd.read_excel(dict_file['file_name'], header=dict_file['head']-1, sheet_name=dict_file['sheet_name'])

df_data

,序号,工厂,工厂号,完整OE,是否采购,QTY,含税价格,备注
0,1,工厂,1,7H0121407C,NaN,NaN,NaN,NaN
1,2,工厂,2,025121403B/\n025121407A,NaN,NaN,NaN,NaN
2,3,工厂,3,025121403A/\n025121407,NaN,NaN,NaN,NaN
3,4,工厂,4,171121407F,NaN,NaN,NaN,NaN
4,5,工厂,5,8E0121403A\n8E0121403D\n8E0121321,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1098,901,系统,ET87039-Y-4,"17137787039,603-535(带传感器和承压200N的盖子）",NaN,NaN,NaN,YHT调拨专用
1099,902,系统,ET87039-Y-9,17137787039;603-535(带传感器和承压200N的盖子）,NaN,NaN,NaN,YHT调拨专用
1100,903,系统,ET90820-Y-2,1638690820,NaN,NaN,NaN,YHT调拨专用
1101,904,系统,ET91020-Y-4,2218691020,NaN,NaN,NaN,YHT调拨专用


In [6]:
for i in range(len(df_data)):
    df_data.loc[i, dict_file['col_oe']] = normalise_oe(df_data.loc[i, dict_file['col_oe']])

df_data

,序号,工厂,工厂号,完整OE,是否采购,QTY,含税价格,备注
0,1,工厂,1,7H0121407C,NaN,NaN,NaN,NaN
1,2,工厂,2,025121403B;025121407A,NaN,NaN,NaN,NaN
2,3,工厂,3,025121403A;025121407,NaN,NaN,NaN,NaN
3,4,工厂,4,171121407F,NaN,NaN,NaN,NaN
4,5,工厂,5,8E0121403A;8E0121403D;8E0121321,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1098,901,系统,ET87039-Y-4,17137787039;603535,NaN,NaN,NaN,YHT调拨专用
1099,902,系统,ET87039-Y-9,17137787039;603535,NaN,NaN,NaN,YHT调拨专用
1100,903,系统,ET90820-Y-2,1638690820,NaN,NaN,NaN,YHT调拨专用
1101,904,系统,ET91020-Y-4,2218691020,NaN,NaN,NaN,YHT调拨专用


# 2. 查重

In [7]:
list_chachong = []
for i in range(len(df_data)):
    print(i)
    data_oes = df_data.loc[i, dict_file['col_oe']].split(';')
    
    list_field = []
    for j in range(len(df_data)):
        field_oes = df_data.loc[j, dict_file['col_oe']].split(';')
        
        if i == j:
            continue
        
        for k in range(len(data_oes)):
            if data_oes[k] in field_oes:
                if dict_file['col_oe'] == dict_file['col_field']:
                    list_field.append(data_oes[k])
                else:
                    list_field.append(df_data.loc[j, dict_file['col_field']])
                break
    
    set_field = set(list_field)
    if len(set_field) == 0:
        list_chachong.append('')
    else:
        set_field = [str(i) for i in set_field]
        list_chachong.append(';'.join(set_field))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
df_data['查重'] = list_chachong

df_data

,序号,工厂,工厂号,完整OE,是否采购,QTY,含税价格,备注,查重
0,1,工厂,1,7H0121407C,NaN,NaN,NaN,NaN,7H0121407C
1,2,工厂,2,025121403B;025121407A,NaN,NaN,NaN,NaN,025121403B
2,3,工厂,3,025121403A;025121407,NaN,NaN,NaN,NaN,025121403A
3,4,工厂,4,171121407F,NaN,NaN,NaN,NaN,171121407F
4,5,工厂,5,8E0121403A;8E0121403D;8E0121321,NaN,NaN,NaN,NaN,8E0121403D;8E0121403A
...,...,...,...,...,...,...,...,...,...
1098,901,系统,ET87039-Y-4,17137787039;603535,NaN,NaN,NaN,YHT调拨专用,17137787039
1099,902,系统,ET87039-Y-9,17137787039;603535,NaN,NaN,NaN,YHT调拨专用,17137787039
1100,903,系统,ET90820-Y-2,1638690820,NaN,NaN,NaN,YHT调拨专用,1638690820
1101,904,系统,ET91020-Y-4,2218691020,NaN,NaN,NaN,YHT调拨专用,2218691020


# 3. 保存

In [9]:
import datetime

writer = pd.ExcelWriter(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
df_data.to_excel(writer, index=False)
writer.close()